<a href="https://colab.research.google.com/github/feelpp/book.feelpp.org/blob/master/feelpp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!ls /usr/lib/python3/dist-packages/feelpp/
!echo $PYTHONPATH

_alg.cpython-37m-x86_64-linux-gnu.so	   __pycache__
_core.cpython-37m-x86_64-linux-gnu.so	   test_core.py
_discr.cpython-37m-x86_64-linux-gnu.so	   test_discr.py
_exporter.cpython-37m-x86_64-linux-gnu.so  test_exporter.py
_feelpp.cpython-37m-x86_64-linux-gnu.so    test_mesh.py
__init__.py				   test_remotedata.py
_mesh.cpython-37m-x86_64-linux-gnu.so	   _ts.cpython-37m-x86_64-linux-gnu.so
_models.cpython-37m-x86_64-linux-gnu.so    _vf.cpython-37m-x86_64-linux-gnu.so
/usr/lib/python3/dist-packages/


## Core Test

In [ ]:
import sys
sys.path.insert(0,'/usr/lib/python3/dist-packages')
print(sys.version,sys.version_info)
import feelpp
e=feelpp.Environment(sys.argv)
e.setConfigFile("/usr/share/feelpp/data/testcases/quickstart/cases/triangle/triangle.cfg")
print("Number of processes:",e.numberOfProcessors())
print("isMasterRank:",e.isMasterRank() )

3.7.9 (default, Sep 10 2020, 17:21:15) 
[GCC 8.3.0] sys.version_info(major=3, minor=7, micro=9, releaselevel='final', serial=0)
Number of processes: 1
isMasterRank: True


## Mesh Test

In [ ]:
geo={
    '2':feelpp.download( "github:{repo:feelpp,path:feelpp/quickstart/laplacian/cases/feelpp2d/feelpp2d.geo}", worldComm=feelpp.Environment.worldCommPtr() )[0],
    '3':feelpp.download( "github:{repo:feelpp,path:feelpp/quickstart/laplacian/cases/feelpp3d/feelpp3d.geo}", worldComm=feelpp.Environment.worldCommPtr() )[0]
}


def run( m, geofile ):
    if e.isMasterRank():
        print("mesh dim:", m.dimension())
    
    m=feelpp.load(m,geofile,0.1)
    if e.isMasterRank():
        print("mesh ",m.dimension(),"D nelts:", m.numGlobalElements() )
        print("mesh ",m.dimension(),"D nfaces:", m.numGlobalFaces() )
        print("mesh ",m.dimension(),"D hmin:", m.hMin())
        print("mesh ",m.dimension(),"D havg:", m.hAverage())
        print("mesh ",m.dimension(),"D hmax:", m.hMax())
        print("mesh ",m.dimension(),"D measure:", m.measure())
    
    r = feelpp.elements(m)
    print("mesh elts:", feelpp.nelements(r,True))
    r = feelpp.boundaryfaces(m)
    print("mesh boundary faces:", feelpp.nfaces(r,True))

run( feelpp.mesh(dim=2), geo['2'] )
run( feelpp.mesh(dim=3,realdim=3), geo['3'] )


mesh dim: 2
mesh  2 D nelts: 5134
mesh  2 D nfaces: 8006
mesh  2 D hmin: 0.0763352613189128
mesh  2 D havg: 0.10274873094289755
mesh  2 D hmax: 0.13695416558140516
mesh  2 D measure: 20.79999999999998
mesh elts: 5134
mesh boundary faces: 610
mesh dim: 3
mesh  3 D nelts: 93999
mesh  3 D nfaces: 200578
mesh  3 D hmin: 0.09479024041018817
mesh  3 D havg: 0.16517492232338588
mesh  3 D hmax: 0.2236800800238625
mesh  3 D measure: 20.799999999999894
mesh elts: 93999
mesh boundary faces: 25160


## Function Space Test

In [ ]:
geo={
    '2':feelpp.download( "github:{repo:feelpp,path:feelpp/quickstart/laplacian/cases/feelpp2d/feelpp2d.geo}", worldComm=feelpp.Environment.worldCommPtr() )[0],
    '3':feelpp.download( "github:{repo:feelpp,path:feelpp/quickstart/laplacian/cases/feelpp3d/feelpp3d.geo}", worldComm=feelpp.Environment.worldCommPtr() )[0]
}

def run( m, geo ):
    m2d = feelpp.load(m,geo,0.1)
    print("## ", m2d.dimension(),  "D problem")
    Xh=feelpp.functionSpace(mesh=m2d)

    if e.isMasterRank():
        print("Xh basisname: ", Xh.basisName())
        print("Xh nDof: ", Xh.nDof())
        print("Xh nLocalDof: ", Xh.nLocalDof())
        print("Xh nLocalDofWithGhost: ", Xh.nLocalDofWithGhost())
        print("Xh nLocalDofWithoutGhost: ", Xh.nLocalDofWithoutGhost())

    m3=Xh.mesh()

    assert m3==m2d

    u=Xh.element()
    u.on(range=feelpp.elements(m2d),expr=feelpp.expr("x:x"))

    assert u.functionSpace() == Xh
    assert u.size() == Xh.nDof()

run( feelpp.mesh(dim=2), geo['2'] )
run( feelpp.mesh(dim=3,realdim=3), geo['3'] )


##  2 D problem
Xh basisname:  lagrange
Xh nDof:  2867
Xh nLocalDof:  2867
Xh nLocalDofWithGhost:  2867
Xh nLocalDofWithoutGhost:  2867
##  3 D problem
Xh basisname:  lagrange
Xh nDof:  21739
Xh nLocalDof:  21739
Xh nLocalDofWithGhost:  21739
Xh nLocalDofWithoutGhost:  21739


## Exporter Test

In [ ]:

geo={
    '2':feelpp.download( "github:{repo:feelpp,path:feelpp/quickstart/laplacian/cases/feelpp2d/feelpp2d.geo}", worldComm=feelpp.Environment.worldCommPtr() )[0],
    '3':feelpp.download( "github:{repo:feelpp,path:feelpp/quickstart/laplacian/cases/feelpp3d/feelpp3d.geo}", worldComm=feelpp.Environment.worldCommPtr() )[0]
}

def run( m, geo ):
    mesh = feelpp.load(m,geo,0.1)
    print("## ", mesh.dimension(),  "D problem")
    Xh=feelpp.functionSpace( space="Pch", mesh=mesh, order=1 )
    P0h = feelpp.functionSpace( space="Pdh", mesh=mesh, order=0 )
    #u=Xh.elementFromExpr("{sin(2*pi*x)*cos(pi*y)}:x:y")
    u=Xh.element()
    u.on(range=feelpp.elements(mesh),expr=feelpp.expr("x*x+y:x:y"))

    for  norm in [feelpp.mean,feelpp.normL2,feelpp.normH1]:
        print(norm.__name__,"(u)=",norm(range=feelpp.elements(mesh),expr=u))
    e = feelpp.exporter(mesh=mesh,name="feelpp"+str(m.dimension())+"d")
    e.addScalar("un", 1.)
    e.addP1c("u",u)
    #e.addP1c("v",feelpp.expr(x:))
    e.addP0d("pid",feelpp.pid( P0h ))
    e.save()

run( feelpp.mesh( dim=2 ), geo['2'] )
run( feelpp.mesh( dim=3, realdim=3 ), geo['3'] )


##  2 D problem
mean (u)= [24.01591361]
normL2 (u)= 145.32682386943628
normH1 (u)= 151.83739662047932
##  3 D problem
mean (u)= [24.01661583]
normL2 (u)= 145.32924129974887
normH1 (u)= 151.839708610157


## Interpolation Convergence Test

In [ ]:
geo = feelpp.download( "github:{repo:feelpp,path:feelpp/quickstart/laplacian/cases/feelpp2d/feelpp2d.geo}", worldComm=feelpp.Environment.worldCommPtr() )[0]

mesh = feelpp.load(m,geo,0.1)

